In [2]:
import pandas as pd
import numpy as np
import gc
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from tqdm.auto import tqdm
warnings.filterwarnings('ignore')

# Enable tqdm for pandas operations
tqdm.pandas()

In [3]:
class Config:
    # Paths (adjust based on your Kaggle dataset location)
    DATA_PATH = Path('/Users/raghu/Desktop/Quarter_1/CSE_258R/assignment2/h-and-m-personalized-fashion-recommendations')
    OUTPUT_PATH = Path('/Users/raghu/Desktop/Quarter_1/CSE_258R/assignment2/fashion_recommender')
    
    # Temporal configuration
    N_TRAIN_WEEKS = 12  # Number of weeks for training
    N_VAL_WEEKS = 1     # Validation week
    
    # Sampling configuration
    MIN_ITEM_PURCHASES = 2  # Minimum purchases for an item to be included
    USER_SAMPLE_RATE = None  # None = use active user strategy (recommended)
    
    # Memory optimization
    CHUNK_SIZE = 500000  # Process transactions in chunks
    
    # Random seed
    RANDOM_STATE = 42

config = Config()
config.OUTPUT_PATH.mkdir(exist_ok=True)

In [4]:
# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def reduce_mem_usage(df, verbose=True):
    """
    Reduce memory usage of a dataframe by optimizing dtypes
    """
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in tqdm(df.columns, desc="Optimizing dtypes", disable=not verbose):
        col_type = df[col].dtype
        
        # Skip datetime columns
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            continue
        
        # Skip object/string columns (convert to category instead)
        if col_type == object:
            num_unique_values = len(df[col].unique())
            num_total_values = len(df[col])
            if num_unique_values / num_total_values < 0.5:  # If less than 50% unique, convert to category
                df[col] = df[col].astype('category')
            continue
        
        # Optimize numeric columns
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            # Integer columns
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            # Float columns
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    
    end_mem = df.memory_usage().sum() / 1024**2
    
    if verbose:
        print(f'Memory usage decreased from {start_mem:.2f} MB to {end_mem:.2f} MB '
              f'({100 * (start_mem - end_mem) / start_mem:.1f}% reduction)')
    
    return df

def print_section(title):
    """Pretty print section headers"""
    print("\n" + "="*80)
    print(f"  {title}")
    print("="*80)

# ============================================================================
# STEP 1: LOAD AND EXPLORE DATA
# ============================================================================

print_section("STEP 1: LOADING DATA")

# Load transactions
print("Loading transactions...")
transactions = pd.read_csv(
    config.DATA_PATH / 'transactions_train.csv',
    dtype={
        'article_id': 'int32',
        'price': 'float32',
        'sales_channel_id': 'int8'
    },
    parse_dates=['t_dat']
)
print(f"✓ Loaded {len(transactions):,} transactions")
print(f"  Date range: {transactions['t_dat'].min()} to {transactions['t_dat'].max()}")
print(f"  Unique customers: {transactions['customer_id'].nunique():,}")
print(f"  Unique articles: {transactions['article_id'].nunique():,}")

# Load customers
print("\nLoading customers...")
customers = pd.read_csv(
    config.DATA_PATH / 'customers.csv',
    dtype={
        'FN': 'float32',
        'Active': 'float32',
        'age': 'float32'
    }
)
customers = reduce_mem_usage(customers, verbose=False)
print(f"✓ Loaded {len(customers):,} customers")

# Load articles
print("\nLoading articles...")
articles = pd.read_csv(
    config.DATA_PATH / 'articles.csv',
    dtype={'article_id': 'int32'}
)
articles = reduce_mem_usage(articles, verbose=False)
print(f"✓ Loaded {len(articles):,} articles")

# ============================================================================
# STEP 2: TEMPORAL WINDOW SELECTION
# ============================================================================

print_section("STEP 2: SELECTING TEMPORAL WINDOW")

# Get the last date in transactions
max_date = transactions['t_dat'].max()
print(f"Last transaction date: {max_date}")

# Calculate cutoff dates
val_end_date = max_date
val_start_date = val_end_date - timedelta(weeks=config.N_VAL_WEEKS)
train_end_date = val_start_date - timedelta(days=1)
train_start_date = train_end_date - timedelta(weeks=config.N_TRAIN_WEEKS)

print(f"\nTemporal splits:")
print(f"  Training:   {train_start_date.date()} to {train_end_date.date()} ({config.N_TRAIN_WEEKS} weeks)")
print(f"  Validation: {val_start_date.date()} to {val_end_date.date()} ({config.N_VAL_WEEKS} week)")

# Filter transactions to our window
total_weeks = config.N_TRAIN_WEEKS + config.N_VAL_WEEKS
window_start = max_date - timedelta(weeks=total_weeks)

print(f"\nFiltering transactions from {window_start.date()} onwards...")
transactions = transactions[transactions['t_dat'] >= window_start].copy()
print(f"✓ Retained {len(transactions):,} transactions ({len(transactions)/1e6:.2f}M)")

# Add week number (relative to window start)
transactions['week'] = ((transactions['t_dat'] - window_start).dt.days // 7).astype(np.int8)
print(f"  Week range: {transactions['week'].min()} to {transactions['week'].max()}")

# Split into train and validation
train_transactions = transactions[transactions['t_dat'] <= train_end_date].copy()
val_transactions = transactions[transactions['t_dat'] > train_end_date].copy()

print(f"\nDataset split:")
print(f"  Training transactions: {len(train_transactions):,}")
print(f"  Validation transactions: {len(val_transactions):,}")

del transactions
gc.collect()


  STEP 1: LOADING DATA
Loading transactions...
✓ Loaded 31,788,324 transactions
  Date range: 2018-09-20 00:00:00 to 2020-09-22 00:00:00
  Unique customers: 1,362,281
  Unique articles: 104,547

Loading customers...
✓ Loaded 1,371,980 customers

Loading articles...
✓ Loaded 105,542 articles

  STEP 2: SELECTING TEMPORAL WINDOW
Last transaction date: 2020-09-22 00:00:00

Temporal splits:
  Training:   2020-06-22 to 2020-09-14 (12 weeks)
  Validation: 2020-09-15 to 2020-09-22 (1 week)

Filtering transactions from 2020-06-23 onwards...
✓ Retained 3,981,458 transactions (3.98M)
  Week range: 0 to 13

Dataset split:
  Training transactions: 3,715,094
  Validation transactions: 266,364


28

In [5]:
# ============================================================================
# STEP 3: SMART USER SAMPLING
# ============================================================================

print_section("STEP 3: SMART USER SAMPLING")

# Strategy: Keep ALL users who purchased in validation week + active users in train

# Get users who purchased in validation week (these are who we need to predict for)
val_users = set(val_transactions['customer_id'].unique())
print(f"Users in validation week: {len(val_users):,}")

# Get active users in last 3 weeks of training (recent activity indicates likely future purchase)
last_3_weeks_date = train_end_date - timedelta(weeks=3)
active_train_users = set(
    train_transactions[train_transactions['t_dat'] >= last_3_weeks_date]['customer_id'].unique()
)
print(f"Active users in last 3 weeks of training: {len(active_train_users):,}")

# Combine: validation users + active training users
selected_users = val_users.union(active_train_users)
print(f"\nTotal selected users: {len(selected_users):,}")
print(f"  Reduction: {100 * (1 - len(selected_users)/customers['customer_id'].nunique()):.1f}%")

# Filter transactions to selected users
train_transactions = train_transactions[train_transactions['customer_id'].isin(selected_users)].copy()
val_transactions = val_transactions[val_transactions['customer_id'].isin(selected_users)].copy()

print(f"\nAfter user filtering:")
print(f"  Training transactions: {len(train_transactions):,}")
print(f"  Validation transactions: {len(val_transactions):,}")

# Filter customers table
customers = customers[customers['customer_id'].isin(selected_users)].copy()
print(f"  Customers retained: {len(customers):,}")


  STEP 3: SMART USER SAMPLING
Users in validation week: 75,481
Active users in last 3 weeks of training: 200,756

Total selected users: 245,945
  Reduction: 82.1%

After user filtering:
  Training transactions: 2,129,323
  Validation transactions: 266,364
  Customers retained: 245,945


In [6]:
# ============================================================================
# STEP 4: ITEM FILTERING
# ============================================================================

print_section("STEP 4: ITEM FILTERING")

# Count purchases per item in training window
item_counts = train_transactions['article_id'].value_counts()
print(f"Unique items in training: {len(item_counts):,}")

# Keep items with minimum purchases
valid_items = set(item_counts[item_counts >= config.MIN_ITEM_PURCHASES].index)
print(f"Items with >= {config.MIN_ITEM_PURCHASES} purchases: {len(valid_items):,}")

# Also include all items from validation (even if rare in training)
val_items = set(val_transactions['article_id'].unique())
print(f"Items in validation: {len(val_items):,}")

# Combine
selected_items = valid_items.union(val_items)
print(f"\nTotal selected items: {len(selected_items):,}")
print(f"  Reduction: {100 * (1 - len(selected_items)/articles['article_id'].nunique()):.1f}%")

# Filter transactions
train_transactions = train_transactions[train_transactions['article_id'].isin(selected_items)].copy()
val_transactions = val_transactions[val_transactions['article_id'].isin(selected_items)].copy()

print(f"\nAfter item filtering:")
print(f"  Training transactions: {len(train_transactions):,}")
print(f"  Validation transactions: {len(val_transactions):,}")

# Filter articles table
articles = articles[articles['article_id'].isin(selected_items)].copy()
print(f"  Articles retained: {len(articles):,}")


  STEP 4: ITEM FILTERING
Unique items in training: 37,470
Items with >= 2 purchases: 30,967
Items in validation: 18,684

Total selected items: 32,714
  Reduction: 69.0%

After item filtering:
  Training transactions: 2,123,394
  Validation transactions: 266,364
  Articles retained: 32,714


In [7]:
# ============================================================================
# STEP 5: DATA TYPE OPTIMIZATION
# ============================================================================

print_section("STEP 5: MEMORY OPTIMIZATION")

print("Before optimization:")
print(f"  train_transactions: {train_transactions.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"  val_transactions: {val_transactions.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"  customers: {customers.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"  articles: {articles.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Optimize transactions
train_transactions = reduce_mem_usage(train_transactions)
val_transactions = reduce_mem_usage(val_transactions)

# Convert categorical columns
print("\nConverting article categorical columns...")
for col in tqdm(['product_code', 'product_type_no', 'graphical_appearance_no', 
                 'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id',
                 'department_no', 'index_code', 'index_group_no', 'section_no', 'garment_group_no'],
                desc="Article categories"):
    if col in articles.columns:
        articles[col] = articles[col].astype('category')

# Optimize customer categoricals
print("\nConverting customer categorical columns...")
for col in tqdm(['club_member_status', 'fashion_news_frequency', 'postal_code'],
                desc="Customer categories"):
    if col in customers.columns:
        customers[col] = customers[col].astype('category')

print("\nAfter optimization:")
print(f"  train_transactions: {train_transactions.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"  val_transactions: {val_transactions.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"  customers: {customers.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"  articles: {articles.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


  STEP 5: MEMORY OPTIMIZATION
Before optimization:
  train_transactions: 297.68 MB
  val_transactions: 37.34 MB
  customers: 83.26 MB
  articles: 16.11 MB


Optimizing dtypes:   0%|          | 0/6 [00:00<?, ?it/s]

Memory usage decreased from 68.85 MB to 70.52 MB (-2.4% reduction)


Optimizing dtypes:   0%|          | 0/6 [00:00<?, ?it/s]

Memory usage decreased from 8.64 MB to 10.21 MB (-18.2% reduction)

Converting article categorical columns...


Article categories:   0%|          | 0/11 [00:00<?, ?it/s]


Converting customer categorical columns...


Customer categories:   0%|          | 0/3 [00:00<?, ?it/s]


After optimization:
  train_transactions: 94.70 MB
  val_transactions: 18.35 MB
  customers: 83.26 MB
  articles: 16.34 MB


In [8]:
# ============================================================================
# STEP 6: BASIC EDA AND VALIDATION
# ============================================================================

print_section("STEP 6: DATA VALIDATION & EDA")

# Validation checks
print("Data validation:")
print(f"  ✓ No null customer_ids in train: {train_transactions['customer_id'].isnull().sum() == 0}")
print(f"  ✓ No null article_ids in train: {train_transactions['article_id'].isnull().sum() == 0}")
print(f"  ✓ All validation users in training: {val_users.issubset(set(train_transactions['customer_id']))}")

# Basic statistics
print("\nTransaction statistics:")
print(f"  Training period: {train_transactions['t_dat'].min().date()} to {train_transactions['t_dat'].max().date()}")
print(f"  Validation period: {val_transactions['t_dat'].min().date()} to {val_transactions['t_dat'].max().date()}")
print(f"  Avg transactions per user (train): {len(train_transactions) / len(selected_users):.2f}")
print(f"  Avg transactions per item (train): {len(train_transactions) / len(selected_items):.2f}")

# Purchase distribution
print("\nPurchase distribution in validation week:")
val_user_purchases = val_transactions.groupby('customer_id').size()
print(f"  Mean purchases per user: {val_user_purchases.mean():.2f}")
print(f"  Median purchases per user: {val_user_purchases.median():.0f}")
print(f"  Users with 1 purchase: {(val_user_purchases == 1).sum():,} ({100*(val_user_purchases == 1).sum()/len(val_user_purchases):.1f}%)")
print(f"  Users with 2-5 purchases: {((val_user_purchases >= 2) & (val_user_purchases <= 5)).sum():,}")
print(f"  Users with 6+ purchases: {(val_user_purchases >= 6).sum():,}")

# Weekly transaction trend
weekly_counts = train_transactions.groupby('week').size()
print("\nWeekly transaction counts:")
for week, count in weekly_counts.items():
    print(f"  Week {week}: {count:,} transactions")


  STEP 6: DATA VALIDATION & EDA
Data validation:
  ✓ No null customer_ids in train: True
  ✓ No null article_ids in train: True
  ✓ All validation users in training: False

Transaction statistics:
  Training period: 2020-06-23 to 2020-09-14
  Validation period: 2020-09-15 to 2020-09-22
  Avg transactions per user (train): 8.63
  Avg transactions per item (train): 64.91

Purchase distribution in validation week:
  Mean purchases per user: 3.53
  Median purchases per user: 2
  Users with 1 purchase: 21,600 (28.6%)
  Users with 2-5 purchases: 40,847
  Users with 6+ purchases: 13,034

Weekly transaction counts:
  Week 0: 225,490 transactions
  Week 1: 151,983 transactions
  Week 2: 129,666 transactions
  Week 3: 125,384 transactions
  Week 4: 131,386 transactions
  Week 5: 139,736 transactions
  Week 6: 135,173 transactions
  Week 7: 125,580 transactions
  Week 8: 141,604 transactions
  Week 9: 286,430 transactions
  Week 10: 265,769 transactions
  Week 11: 265,193 transactions


In [9]:
# ============================================================================
# STEP 7: CREATE VALIDATION GROUND TRUTH
# ============================================================================

print_section("STEP 7: CREATING VALIDATION GROUND TRUTH")

# Create validation ground truth (actual purchases in validation week)
val_ground_truth = (
    val_transactions
    .groupby('customer_id')['article_id']
    .apply(list)
    .reset_index()
    .rename(columns={'article_id': 'purchased_articles'})
)

print(f"Validation ground truth:")
print(f"  Users: {len(val_ground_truth):,}")
print(f"  Total purchases: {val_ground_truth['purchased_articles'].apply(len).sum():,}")
print(f"  Avg purchases per user: {val_ground_truth['purchased_articles'].apply(len).mean():.2f}")


  STEP 7: CREATING VALIDATION GROUND TRUTH
Validation ground truth:
  Users: 75,481
  Total purchases: 266,364
  Avg purchases per user: 3.53


In [10]:
# ============================================================================
# STEP 8: SAVE PROCESSED DATA
# ============================================================================

print_section("STEP 8: SAVING PROCESSED DATA")

# Save to parquet (much more efficient than CSV)
print("Saving files...")

files_to_save = [
    (train_transactions, 'train_transactions.parquet'),
    (val_transactions, 'val_transactions.parquet'),
    (customers, 'customers.parquet'),
    (articles, 'articles.parquet'),
    (val_ground_truth, 'val_ground_truth.parquet')
]

for df, filename in tqdm(files_to_save, desc="Saving files"):
    df.to_parquet(config.OUTPUT_PATH / filename, index=False)
    print(f"  ✓ {filename} ({len(df):,} rows)")

# Save metadata
metadata = {
    'train_start_date': str(train_start_date.date()),
    'train_end_date': str(train_end_date.date()),
    'val_start_date': str(val_start_date.date()),
    'val_end_date': str(val_end_date.date()),
    'n_users': len(selected_users),
    'n_items': len(selected_items),
    'n_train_transactions': len(train_transactions),
    'n_val_transactions': len(val_transactions),
}

import json
with open(config.OUTPUT_PATH / 'metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"  ✓ metadata.json")


  STEP 8: SAVING PROCESSED DATA
Saving files...


Saving files:   0%|          | 0/5 [00:00<?, ?it/s]

  ✓ train_transactions.parquet (2,123,394 rows)
  ✓ val_transactions.parquet (266,364 rows)
  ✓ customers.parquet (245,945 rows)
  ✓ articles.parquet (32,714 rows)
  ✓ val_ground_truth.parquet (75,481 rows)
  ✓ metadata.json


In [11]:
# ============================================================================
# SUMMARY
# ============================================================================

print_section("PREPROCESSING COMPLETE!")

print("\nFinal dataset summary:")
print(f"  📅 Training weeks: {config.N_TRAIN_WEEKS}")
print(f"  📅 Validation weeks: {config.N_VAL_WEEKS}")
print(f"  👥 Users: {len(selected_users):,}")
print(f"  🛍️  Items: {len(selected_items):,}")
print(f"  📊 Train transactions: {len(train_transactions):,}")
print(f"  📊 Val transactions: {len(val_transactions):,}")
print(f"  💾 Total disk space: {sum((config.OUTPUT_PATH / f).stat().st_size for f in ['train_transactions.parquet', 'val_transactions.parquet', 'customers.parquet', 'articles.parquet', 'val_ground_truth.parquet']) / 1024**2:.2f} MB")

print("\n✅ Ready for Stage 2: Recall Strategies!")
print("\nNext steps:")
print("  1. Review the saved files in /kaggle/working/")
print("  2. Check metadata.json for dataset info")
print("  3. Proceed to Stage 2 when ready")


  PREPROCESSING COMPLETE!

Final dataset summary:
  📅 Training weeks: 12
  📅 Validation weeks: 1
  👥 Users: 245,945
  🛍️  Items: 32,714
  📊 Train transactions: 2,123,394
  📊 Val transactions: 266,364
  💾 Total disk space: 83.13 MB

✅ Ready for Stage 2: Recall Strategies!

Next steps:
  1. Review the saved files in /kaggle/working/
  2. Check metadata.json for dataset info
  3. Proceed to Stage 2 when ready


In [2]:
import pandas as pd
import numpy as np
import gc
import os
import psutil
from pathlib import Path
from collections import defaultdict, Counter
from datetime import datetime, timedelta
import warnings
from tqdm.auto import tqdm
import pickle

warnings.filterwarnings('ignore')

In [3]:
# ============================================================================
# MEMORY MONITORING
# ============================================================================

def get_memory_usage():
    """Get current memory usage in GB"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024**3

def print_memory():
    """Print current memory usage"""
    mem = get_memory_usage()
    print(f"  💾 Memory: {mem:.2f} GB")

def force_garbage_collection():
    """Aggressive garbage collection"""
    gc.collect()
    gc.collect()
    gc.collect()

In [4]:
class Config:
    # Paths
    DATA_PATH = Path('/Users/raghu/Desktop/Quarter_1/CSE_258R/assignment2/fashion_recommender')
    OUTPUT_PATH = Path('/Users/raghu/Desktop/Quarter_1/CSE_258R/assignment2/fashion_recommender_candidate_generation')
    
    # Recall configuration - REDUCED for memory
    N_REPURCHASE_CANDIDATES = 25  # Reduced from 30
    N_POPULARITY_CANDIDATES = 25  # Reduced from 30
    N_COPURCHASE_CANDIDATES = 15  # Reduced from 20
    N_USERKNN_CANDIDATES = 15     # Reduced from 20
    N_CATEGORY_CANDIDATES = 15    # Reduced from 20
    
    # Processing parameters
    USER_CHUNK_SIZE = 1000  # Process users in chunks
    BASKET_CHUNK_SIZE = 5000  # Process baskets in chunks
    
    # EMERGENCY MODE: Use only recent data for repurchase
    USE_RECENT_ONLY_REPURCHASE = True  # Set to True if kernel keeps crashing
    REPURCHASE_RECENT_WEEKS = 8  # Only use last 8 weeks for repurchase
    
    # Item-to-Item CF parameters
    MIN_ITEM_SUPPORT = 3
    MAX_ITEM_NEIGHBORS = 30  # Reduced from 50
    
    # User-KNN parameters (ONLY for validation users)
    N_SIMILAR_USERS = 20  # Reduced from 30
    MIN_COMMON_ITEMS = 2
    
    # Time decay
    REPURCHASE_DECAY_RATE = 0.05
    POPULARITY_WINDOW_WEEKS = 2
    
    RANDOM_STATE = 42

config = Config()

In [5]:
# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def print_section(title):
    print("\n" + "="*80)
    print(f"  {title}")
    print("="*80 + "\n")

def time_decay_score(days_ago, decay_rate=0.05):
    """Vectorized time decay"""
    return np.exp(-decay_rate * days_ago)

In [6]:
# ============================================================================
# LOAD DATA
# ============================================================================

print_section("LOADING DATA")

print("Loading data files...")
train_transactions = pd.read_parquet(config.DATA_PATH / 'train_transactions.parquet')
val_ground_truth = pd.read_parquet(config.DATA_PATH / 'val_ground_truth.parquet')
articles = pd.read_parquet(config.DATA_PATH / 'articles.parquet')

print(f"✓ Train transactions: {len(train_transactions):,}")
print_memory()

all_users = train_transactions['customer_id'].unique()
all_items = train_transactions['article_id'].unique()
val_users = set(val_ground_truth['customer_id'].unique())
max_date = train_transactions['t_dat'].max()

print(f"✓ Users: {len(all_users):,}, Items: {len(all_items):,}, Val users: {len(val_users):,}")


  LOADING DATA

Loading data files...
✓ Train transactions: 2,123,394
  💾 Memory: 0.32 GB
✓ Users: 224,306, Items: 31,541, Val users: 75,481


In [7]:
# ============================================================================
# STRATEGY 1: REPURCHASE - CHUNKED PROCESSING
# ============================================================================

print_section("STRATEGY 1: REPURCHASE (CHUNKED)")

print("Processing in chunks to save memory...")
repurchase_chunks = []

# Split users into chunks
user_chunks = np.array_split(all_users, max(1, len(all_users) // config.USER_CHUNK_SIZE))

for i, user_chunk in enumerate(tqdm(user_chunks, desc="User chunks")):
    # Filter transactions for this chunk
    chunk_trans = train_transactions[
        train_transactions['customer_id'].isin(user_chunk)
    ].copy()
    
    # Get last purchase per user-item
    user_item_last = (
        chunk_trans
        .groupby(['customer_id', 'article_id'], as_index=False)['t_dat']
        .max()
    )
    
    # Calculate scores (vectorized) - with NaN handling
    user_item_last['days_ago'] = (max_date - user_item_last['t_dat']).dt.days
    
    # Drop any NaN values before converting to int
    user_item_last = user_item_last.dropna(subset=['days_ago'])
    
    # Now safe to convert to int
    user_item_last['days_ago'] = user_item_last['days_ago'].astype(np.int16)
    user_item_last['repurchase_score'] = time_decay_score(
        user_item_last['days_ago'].values, 
        config.REPURCHASE_DECAY_RATE
    ).astype(np.float32)
    
    # Get top N per user
    top_candidates = (
        user_item_last
        .sort_values(['customer_id', 'repurchase_score'], ascending=[True, False])
        .groupby('customer_id', as_index=False)
        .head(config.N_REPURCHASE_CANDIDATES)
        [['customer_id', 'article_id', 'repurchase_score']]
    )
    
    repurchase_chunks.append(top_candidates)
    
    # Clean up
    del chunk_trans, user_item_last, top_candidates
    force_garbage_collection()

# Combine chunks
repurchase_candidates = pd.concat(repurchase_chunks, ignore_index=True)
del repurchase_chunks
force_garbage_collection()

print(f"✓ Generated {len(repurchase_candidates):,} repurchase candidates")
print_memory()

# Save intermediate result
repurchase_candidates.to_parquet(config.OUTPUT_PATH / 'temp_repurchase.parquet', index=False)
del repurchase_candidates
force_garbage_collection()


  STRATEGY 1: REPURCHASE (CHUNKED)

Processing in chunks to save memory...


User chunks:   0%|          | 0/224 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# ============================================================================
# STRATEGY 2: POPULARITY
# ============================================================================

print_section("STRATEGY 2: POPULARITY")

cutoff_date = max_date - timedelta(weeks=config.POPULARITY_WINDOW_WEEKS)
recent_trans = train_transactions[train_transactions['t_dat'] >= cutoff_date].copy()

print(f"Using {len(recent_trans):,} recent transactions")

# Vectorized calculations
recent_trans['days_ago'] = (max_date - recent_trans['t_dat']).dt.days

# Drop NaN values
recent_trans = recent_trans.dropna(subset=['days_ago'])

# Convert to int
recent_trans['days_ago'] = recent_trans['days_ago'].astype(np.int16)
recent_trans['weight'] = time_decay_score(recent_trans['days_ago'].values, 0.1).astype(np.float32)

# Aggregate
item_popularity = (
    recent_trans
    .groupby('article_id', as_index=False)
    .agg({'weight': 'sum', 'customer_id': 'nunique'})
    .rename(columns={'weight': 'weighted_purchases', 'customer_id': 'unique_buyers'})
)

item_popularity['popularity_score'] = (
    0.7 * item_popularity['weighted_purchases'] + 
    0.3 * item_popularity['unique_buyers']
)
item_popularity['popularity_score'] = (
    item_popularity['popularity_score'] / item_popularity['popularity_score'].max()
).astype(np.float32)

# Get top items
top_items = item_popularity.nlargest(config.N_POPULARITY_CANDIDATES, 'popularity_score')

print(f"✓ Top {len(top_items)} popular items")

# Create candidates - CHUNKED
print("Creating popularity candidates in chunks...")
pop_chunks = []

for user_chunk in tqdm(np.array_split(all_users, 20), desc="Popularity chunks"):
    chunk_df = pd.DataFrame({
        'customer_id': np.repeat(user_chunk, len(top_items)),
        'article_id': np.tile(top_items['article_id'].values, len(user_chunk))
    })
    
    rank_penalty = np.tile(1 - np.arange(len(top_items)) * 0.01, len(user_chunk))
    scores = np.tile(top_items['popularity_score'].values, len(user_chunk))
    chunk_df['popularity_score'] = (scores * rank_penalty).astype(np.float32)
    
    pop_chunks.append(chunk_df)

popularity_candidates = pd.concat(pop_chunks, ignore_index=True)
del pop_chunks, recent_trans
force_garbage_collection()

print(f"✓ Generated {len(popularity_candidates):,} popularity candidates")
print_memory()

# Save
popularity_candidates.to_parquet(config.OUTPUT_PATH / 'temp_popularity.parquet', index=False)
item_popularity.to_parquet(config.OUTPUT_PATH / 'item_popularity.parquet', index=False)
del popularity_candidates
force_garbage_collection()

In [ ]:
# ============================================================================
# STRATEGY 3: CO-PURCHASE (Item-to-Item CF)
# ============================================================================

print_section("STRATEGY 3: CO-PURCHASE (Item-to-Item CF)")

print("Building co-purchase matrix...")

# Create item-to-item co-purchase matrix
# Group by transaction/basket (same user, same day)
train_transactions['basket_id'] = (
    train_transactions['customer_id'].astype(str) + '_' + 
    train_transactions['t_dat'].astype(str)
)

# Get baskets with multiple items
basket_items = (
    train_transactions
    .groupby('basket_id')['article_id']
    .apply(list)
    .reset_index()
)

# Filter baskets with at least 2 items
basket_items = basket_items[basket_items['article_id'].apply(len) >= 2]
print(f"  Baskets with 2+ items: {len(basket_items):,}")

# Build co-purchase counts
print("Computing co-purchase frequencies...")
copurchase_counts = defaultdict(lambda: defaultdict(int))

for items in tqdm(basket_items['article_id'], desc="Processing baskets"):
    # For each pair of items in the basket
    for i in range(len(items)):
        for j in range(i + 1, len(items)):
            item1, item2 = items[i], items[j]
            copurchase_counts[item1][item2] += 1
            copurchase_counts[item2][item1] += 1

print(f"✓ Built co-purchase matrix for {len(copurchase_counts):,} items")

# Convert to item-to-item similarity scores
print("Computing item-to-item similarity scores...")
item_to_items = {}

for item1 in tqdm(copurchase_counts.keys(), desc="Computing similarities"):
    # Get co-purchased items
    copurchased = copurchase_counts[item1]
    
    # Filter by minimum support
    copurchased = {
        item2: count 
        for item2, count in copurchased.items() 
        if count >= config.MIN_ITEM_SUPPORT
    }
    
    if copurchased:
        # Sort by count and take top K
        top_items = sorted(
            copurchased.items(), 
            key=lambda x: x[1], 
            reverse=True
        )[:config.MAX_ITEM_NEIGHBORS]
        
        # Normalize scores
        max_count = top_items[0][1]
        item_to_items[item1] = [
            (item2, count / max_count) 
            for item2, count in top_items
        ]

print(f"✓ Computed similarities for {len(item_to_items):,} items")

# Generate co-purchase candidates for each user
print("Generating co-purchase candidates...")
copurchase_candidates = []

# Get recent purchases for each user (last 10)
user_recent_items = (
    train_transactions
    .sort_values('t_dat', ascending=False)
    .groupby('customer_id')['article_id']
    .apply(lambda x: list(x.unique()[:10]))
    .to_dict()
)

for user in tqdm(all_users, desc="User co-purchase recommendations"):
    if user not in user_recent_items:
        continue
    
    user_items = user_recent_items[user]
    candidate_scores = defaultdict(float)
    
    # Aggregate scores from all user's items
    for user_item in user_items:
        if user_item in item_to_items:
            for similar_item, score in item_to_items[user_item]:
                if similar_item not in user_items:  # Don't recommend already purchased
                    candidate_scores[similar_item] += score
    
    # Get top N candidates
    if candidate_scores:
        top_candidates = sorted(
            candidate_scores.items(), 
            key=lambda x: x[1], 
            reverse=True
        )[:config.N_COPURCHASE_CANDIDATES]
        
        for item, score in top_candidates:
            copurchase_candidates.append({
                'customer_id': user,
                'article_id': item,
                'copurchase_score': score
            })

copurchase_candidates = pd.DataFrame(copurchase_candidates)
print(f"✓ Generated {len(copurchase_candidates):,} co-purchase candidates")
print(f"  Users with candidates: {copurchase_candidates['customer_id'].nunique():,}")

In [ ]:
# ============================================================================
# STRATEGY 4: USER-KNN COLLABORATIVE FILTERING
# ============================================================================

print_section("STRATEGY 4: USER-KNN COLLABORATIVE FILTERING")

print("Building user-item matrix...")

# Create sparse user-item matrix (binary: 1 if purchased, 0 otherwise)
# Map users and items to indices
user_to_idx = {user: idx for idx, user in enumerate(all_users)}
item_to_idx = {item: idx for idx, item in enumerate(all_items)}

# Create matrix
n_users = len(all_users)
n_items = len(all_items)

print(f"  Matrix size: {n_users:,} users x {n_items:,} items")

# Use last 4 weeks for user similarity (more recent = more relevant)
recent_date = max_date - timedelta(weeks=4)
recent_user_items = train_transactions[train_transactions['t_dat'] >= recent_date].copy()

user_item_matrix = lil_matrix((n_users, n_items), dtype=np.int8)

print("Populating user-item matrix...")
for _, row in tqdm(recent_user_items.iterrows(), total=len(recent_user_items), desc="Building matrix"):
    user_idx = user_to_idx[row['customer_id']]
    item_idx = item_to_idx[row['article_id']]
    user_item_matrix[user_idx, item_idx] = 1

# Convert to CSR for efficient operations
user_item_matrix = user_item_matrix.tocsr()
print(f"✓ Matrix density: {user_item_matrix.nnz / (n_users * n_items) * 100:.4f}%")

# Compute user-user similarity (only for validation users to save memory)
print(f"Computing user similarities for {len(val_users):,} validation users...")

val_user_indices = [user_to_idx[user] for user in val_users if user in user_to_idx]
val_user_matrix = user_item_matrix[val_user_indices]

# Normalize rows
val_user_matrix_norm = normalize(val_user_matrix, norm='l2', axis=1)
user_item_matrix_norm = normalize(user_item_matrix, norm='l2', axis=1)

# Compute similarity (batch processing to avoid memory issues)
print("Computing cosine similarities...")
batch_size = 1000
userknn_candidates = []

for i in tqdm(range(0, len(val_user_indices), batch_size), desc="Similarity batches"):
    batch_indices = val_user_indices[i:i+batch_size]
    batch_matrix = val_user_matrix_norm[i:i+batch_size]
    
    # Compute similarity with all users
    similarities = cosine_similarity(batch_matrix, user_item_matrix_norm)
    
    # For each user in batch
    for j, user_idx in enumerate(batch_indices):
        user = all_users[user_idx]
        user_sims = similarities[j]
        
        # Get top similar users (exclude self)
        similar_user_indices = np.argsort(user_sims)[::-1][1:config.N_SIMILAR_USERS+1]
        
        # Get items purchased by similar users
        candidate_scores = defaultdict(float)
        user_purchased = set(
            train_transactions[train_transactions['customer_id'] == user]['article_id']
        )
        
        for sim_user_idx in similar_user_indices:
            sim_score = user_sims[sim_user_idx]
            if sim_score < 0.01:  # Skip very dissimilar users
                continue
            
            sim_user = all_users[sim_user_idx]
            sim_user_items = train_transactions[
                train_transactions['customer_id'] == sim_user
            ]['article_id'].unique()
            
            for item in sim_user_items:
                if item not in user_purchased:
                    candidate_scores[item] += sim_score
        
        # Get top N candidates
        if candidate_scores:
            top_candidates = sorted(
                candidate_scores.items(), 
                key=lambda x: x[1], 
                reverse=True
            )[:config.N_USERKNN_CANDIDATES]
            
            for item, score in top_candidates:
                userknn_candidates.append({
                    'customer_id': user,
                    'article_id': item,
                    'userknn_score': score
                })

userknn_candidates = pd.DataFrame(userknn_candidates)
print(f"✓ Generated {len(userknn_candidates):,} user-KNN candidates")

# Clean up memory
del user_item_matrix, val_user_matrix, val_user_matrix_norm, user_item_matrix_norm
gc.collect()

In [ ]:
# ============================================================================
# STRATEGY 5: CATEGORY-BASED RECOMMENDATIONS
# ============================================================================

print_section("STRATEGY 5: CATEGORY-BASED RECOMMENDATIONS")

print("Computing user category preferences...")

# Get user's category preferences
user_categories = (
    train_transactions
    .merge(articles[['article_id', 'product_type_no', 'product_group_name']], on='article_id')
    .groupby(['customer_id', 'product_type_no'])
    .size()
    .reset_index(name='count')
)

# Get top 3 categories per user
user_top_categories = (
    user_categories
    .sort_values(['customer_id', 'count'], ascending=[True, False])
    .groupby('customer_id')
    .head(3)
)

print(f"✓ Computed preferences for {user_top_categories['customer_id'].nunique():,} users")

# Get popular items per category
category_popular_items = (
    train_transactions[train_transactions['t_dat'] >= cutoff_date]
    .merge(articles[['article_id', 'product_type_no']], on='article_id')
    .groupby(['product_type_no', 'article_id'])
    .size()
    .reset_index(name='count')
    .sort_values(['product_type_no', 'count'], ascending=[True, False])
    .groupby('product_type_no')
    .head(10)
)

print("Generating category-based candidates...")
category_candidates = []

for _, row in tqdm(user_top_categories.iterrows(), total=len(user_top_categories), desc="Category recommendations"):
    user = row['customer_id']
    category = row['product_type_no']
    
    # Get popular items in this category
    category_items = category_popular_items[
        category_popular_items['product_type_no'] == category
    ]['article_id'].tolist()
    
    # Get user's purchased items
    user_items = set(
        train_transactions[train_transactions['customer_id'] == user]['article_id']
    )
    
    # Recommend items not yet purchased
    for rank, item in enumerate(category_items):
        if item not in user_items and rank < config.N_CATEGORY_CANDIDATES:
            category_candidates.append({
                'customer_id': user,
                'article_id': item,
                'category_score': 1.0 / (rank + 1)  # Rank-based score
            })

category_candidates = pd.DataFrame(category_candidates)
print(f"✓ Generated {len(category_candidates):,} category-based candidates")

In [ ]:
# ============================================================================
# COMBINE ALL CANDIDATES
# ============================================================================

print_section("COMBINING ALL RECALL STRATEGIES")

print("Merging candidates from all strategies...")

# Merge all candidates
all_candidates = repurchase_candidates.copy()

# Merge popularity
all_candidates = all_candidates.merge(
    popularity_candidates,
    on=['customer_id', 'article_id'],
    how='outer'
)

# Merge co-purchase
all_candidates = all_candidates.merge(
    copurchase_candidates,
    on=['customer_id', 'article_id'],
    how='outer'
)

# Merge user-KNN
all_candidates = all_candidates.merge(
    userknn_candidates,
    on=['customer_id', 'article_id'],
    how='outer'
)

# Merge category
all_candidates = all_candidates.merge(
    category_candidates,
    on=['customer_id', 'article_id'],
    how='outer'
)

# Fill NaN scores with 0
score_columns = [
    'repurchase_score', 'popularity_score', 'copurchase_score', 
    'userknn_score', 'category_score'
]
all_candidates[score_columns] = all_candidates[score_columns].fillna(0)

print(f"✓ Total unique user-item pairs: {len(all_candidates):,}")

# Count how many strategies recommend each item
all_candidates['n_strategies'] = (all_candidates[score_columns] > 0).sum(axis=1)

print("\nCandidate statistics:")
print(f"  Candidates per user: {len(all_candidates) / len(all_users):.2f}")
print(f"  Avg strategies per candidate: {all_candidates['n_strategies'].mean():.2f}")
print("\n  Candidates by number of strategies:")
for n in sorted(all_candidates['n_strategies'].unique()):
    count = (all_candidates['n_strategies'] == n).sum()
    pct = count / len(all_candidates) * 100
    print(f"    {n} strategies: {count:,} ({pct:.1f}%)")

In [ ]:
# ============================================================================
# EVALUATE RECALL ON VALIDATION SET
# ============================================================================

print_section("EVALUATING RECALL PERFORMANCE")

# Filter to validation users only
val_candidates = all_candidates[all_candidates['customer_id'].isin(val_users)].copy()

print(f"Validation candidates: {len(val_candidates):,}")
print(f"  Avg per user: {len(val_candidates) / len(val_users):.2f}")

# Calculate recall@K for different K values
print("\nRecall@K metrics:")

val_ground_truth_dict = dict(zip(
    val_ground_truth['customer_id'], 
    val_ground_truth['purchased_articles']
))

for k in [12, 50, 100]:
    recalls = []
    
    for user in val_users:
        if user in val_ground_truth_dict:
            true_items = set(val_ground_truth_dict[user])
            
            # Get top K candidates for this user
            user_candidates = (
                val_candidates[val_candidates['customer_id'] == user]
                .nlargest(k, 'n_strategies')  # Simple ranking by coverage
                ['article_id']
                .tolist()
            )
            
            if user_candidates and true_items:
                recall = len(set(user_candidates) & true_items) / len(true_items)
                recalls.append(recall)
            else:
                recalls.append(0)
    
    print(f"  Recall@{k}: {np.mean(recalls):.4f}")

In [ ]:
# ============================================================================
# SAVE CANDIDATES
# ============================================================================

print_section("SAVING CANDIDATES")

print("Saving candidate files...")

# Save all candidates
all_candidates.to_parquet(config.OUTPUT_PATH / 'recall_candidates.parquet', index=False)
print(f"  ✓ recall_candidates.parquet ({len(all_candidates):,} rows, {len(all_candidates) * len(score_columns) * 4 / 1024**2:.2f} MB)")

# Save item-to-item similarity for later use
with open(config.OUTPUT_PATH / 'item_to_items.pkl', 'wb') as f:
    pickle.dump(item_to_items, f)
print(f"  ✓ item_to_items.pkl")

# Save popularity scores
item_popularity.to_parquet(config.OUTPUT_PATH / 'item_popularity.parquet', index=False)
print(f"  ✓ item_popularity.parquet")

# ============================================================================
# SUMMARY
# ============================================================================

print_section("RECALL STAGE COMPLETE!")

print("\nRecall strategy summary:")
print(f"  Strategy 1 - Repurchase: {len(repurchase_candidates):,} candidates")
print(f"  Strategy 2 - Popularity: {len(popularity_candidates):,} candidates")
print(f"  Strategy 3 - Co-Purchase: {len(copurchase_candidates):,} candidates")
print(f"  Strategy 4 - User-KNN: {len(userknn_candidates):,} candidates")
print(f"  Strategy 5 - Category: {len(category_candidates):,} candidates")
print(f"  Total unique pairs: {len(all_candidates):,}")

print("\n✅ Ready for Stage 3: Feature Engineering!")
print("\nNext steps:")
print("  1. Review recall_candidates.parquet")
print("  2. Check recall metrics above")
print("  3. Proceed to Stage 3 when ready")